In [ ]:
!pip install pyodbc

In [ ]:
import pyodbc
import pandas as pd

In [ ]:
connection = pyodbc.connect('Driver={SQL Server};'
                      'Server=smbanalytics.database.windows.net;'
                      'Database=RevenueAttr;'
                      'UID=dgomes;'
                      'PWD=Spr1nger!;'
                      'Trusted_Connection=no;')

cursor = connection.cursor()

In [ ]:
query = "SELECT [CustomerID], [OrgName], [SubsidiaryID], [CountryName] , [SalesMonthID], MonthFullName, C.RevSumDivisionID, RevSumDivisionName, PricingLevelName, ProductFamilyName, \
	   SUM([CommittedLicenseCnt]) AS [CommittedLicenseCnt], SUM([ActualLicenseCnt]) AS [ActualLicenseCnt], SUM([ActualRevenueAmt]) AS [ActualRevenueAmt] \
FROM [ETL].[Source_GraphDB] G \
INNER JOIN [Cube].[DIM:PricingLevelHierarchy] P									  ON G.DetailPricingLevelID = P.DetailPricingLevelID \
INNER JOIN (SELECT DISTINCT Monthid, MonthFullname FROM [Cube].[DIM:SalesDate])SD ON G.SalesMonthID=SD.MonthID \
INNER JOIN [cube].[DIM:RevSumCategory_FY22]	C									  ON C.RevSumCategoryID=G.RevSumCategoryID \
INNER JOIN [cube].[DIM:RevSumDivision] RSD										  ON RSD.RevSumDivisionID=C.RevSumDivisionID \
WHERE AgreementVersionID<>0 \
GROUP BY [CustomerID], [OrgName], [SubsidiaryID], [CountryName], [SalesMonthID], MonthFullName, C.RevSumDivisionID, RevSumDivisionName, PricingLevelName,  ProductFamilyName "

In [ ]:
df = pd.read_sql(query, connection)

In [ ]:
df.to_csv("db.csv")